# Frequency Domain Analysis

This notebook demonstrates comprehensive frequency domain analysis of audio files using the CTC-SpeechRefinement package. We'll explore various spectral features and transformations that are useful for understanding the frequency characteristics of audio signals.

## Setup

First, let's import the necessary libraries and set up the environment.

In [ ]:
# Add the project root to the Python path
import sys
import os
sys.path.append(os.path.abspath('..'))

# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display
import pandas as pd
import seaborn as sns
from IPython.display import Audio, display
import glob
from pathlib import Path

# Import from the project
from ctc_speech_refinement.core.preprocessing.audio import preprocess_audio
from ctc_speech_refinement.core.eda.frequency_domain import analyze_frequency_domain

# Set up plotting
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['figure.dpi'] = 100

## Load Audio Data

Let's load an audio file and examine its basic properties.

In [ ]:
# Define the path to an audio file
audio_file = "../data/test1/test1_01.wav"  # Update this path to your audio file

# Load the audio file
audio_data, sample_rate = librosa.load(audio_file, sr=None)

# Print basic information
print(f"Audio file: {audio_file}")
print(f"Sample rate: {sample_rate} Hz")
print(f"Duration: {len(audio_data) / sample_rate:.2f} seconds")
print(f"Number of samples: {len(audio_data)}")

# Play the audio
display(Audio(audio_data, rate=sample_rate))

## 1. Short-Time Fourier Transform (STFT)

The STFT is a fundamental transform for analyzing the frequency content of audio signals over time.

In [ ]:
# Compute STFT
n_fft = 2048
hop_length = 512
stft = librosa.stft(audio_data, n_fft=n_fft, hop_length=hop_length)
stft_magnitude = np.abs(stft)
stft_db = librosa.amplitude_to_db(stft_magnitude, ref=np.max)

# Plot spectrogram
plt.figure(figsize=(14, 5))
librosa.display.specshow(stft_db, sr=sample_rate, x_axis='time', y_axis='log', hop_length=hop_length)
plt.colorbar(format='%+2.0f dB')
plt.title('Spectrogram (STFT)')
plt.tight_layout()
plt.show()

## 2. Mel Spectrogram

The Mel spectrogram represents the STFT on the Mel scale, which better approximates human auditory perception.

In [ ]:
# Compute Mel spectrogram
n_mels = 128
mel_spectrogram = librosa.feature.melspectrogram(y=audio_data, sr=sample_rate, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels)
mel_spectrogram_db = librosa.power_to_db(mel_spectrogram, ref=np.max)

# Plot Mel spectrogram
plt.figure(figsize=(14, 5))
librosa.display.specshow(mel_spectrogram_db, sr=sample_rate, x_axis='time', y_axis='mel', hop_length=hop_length)
plt.colorbar(format='%+2.0f dB')
plt.title('Mel Spectrogram')
plt.tight_layout()
plt.show()

## 3. Chromagram

The chromagram represents the energy distribution across the 12 pitch classes (chroma).

In [ ]:
# Compute chromagram
chromagram = librosa.feature.chroma_stft(y=audio_data, sr=sample_rate, n_fft=n_fft, hop_length=hop_length)

# Plot chromagram
plt.figure(figsize=(14, 5))
librosa.display.specshow(chromagram, sr=sample_rate, x_axis='time', y_axis='chroma', hop_length=hop_length)
plt.colorbar()
plt.title('Chromagram')
plt.tight_layout()
plt.show()

## 4. Spectral Features

Let's compute and visualize various spectral features.

In [ ]:
# Compute spectral centroid
spectral_centroid = librosa.feature.spectral_centroid(y=audio_data, sr=sample_rate, n_fft=n_fft, hop_length=hop_length)[0]
spectral_centroid_times = librosa.times_like(spectral_centroid, sr=sample_rate, hop_length=hop_length)

# Compute spectral bandwidth
spectral_bandwidth = librosa.feature.spectral_bandwidth(y=audio_data, sr=sample_rate, n_fft=n_fft, hop_length=hop_length)[0]
spectral_bandwidth_times = librosa.times_like(spectral_bandwidth, sr=sample_rate, hop_length=hop_length)

# Compute spectral contrast
spectral_contrast = librosa.feature.spectral_contrast(y=audio_data, sr=sample_rate, n_fft=n_fft, hop_length=hop_length)

# Compute spectral flatness
spectral_flatness = librosa.feature.spectral_flatness(y=audio_data, n_fft=n_fft, hop_length=hop_length)[0]
spectral_flatness_times = librosa.times_like(spectral_flatness, sr=sample_rate, hop_length=hop_length)

# Compute spectral rolloff
spectral_rolloff = librosa.feature.spectral_rolloff(y=audio_data, sr=sample_rate, n_fft=n_fft, hop_length=hop_length)[0]
spectral_rolloff_times = librosa.times_like(spectral_rolloff, sr=sample_rate, hop_length=hop_length)

In [ ]:
# Plot spectral centroid
plt.figure(figsize=(14, 5))
plt.semilogy(spectral_centroid_times, spectral_centroid)
plt.title('Spectral Centroid')
plt.xlabel('Time (s)')
plt.ylabel('Frequency (Hz)')
plt.tight_layout()
plt.show()

In [ ]:
# Plot spectral bandwidth
plt.figure(figsize=(14, 5))
plt.semilogy(spectral_bandwidth_times, spectral_bandwidth)
plt.title('Spectral Bandwidth')
plt.xlabel('Time (s)')
plt.ylabel('Frequency (Hz)')
plt.tight_layout()
plt.show()

In [ ]:
# Plot spectral contrast
plt.figure(figsize=(14, 5))
librosa.display.specshow(spectral_contrast, sr=sample_rate, x_axis='time', hop_length=hop_length)
plt.colorbar()
plt.title('Spectral Contrast')
plt.tight_layout()
plt.show()

In [ ]:
# Plot spectral flatness
plt.figure(figsize=(14, 5))
plt.semilogy(spectral_flatness_times, spectral_flatness)
plt.title('Spectral Flatness')
plt.xlabel('Time (s)')
plt.ylabel('Flatness')
plt.tight_layout()
plt.show()

In [ ]:
# Plot spectral rolloff
plt.figure(figsize=(14, 5))
plt.semilogy(spectral_rolloff_times, spectral_rolloff)
plt.title('Spectral Rolloff')
plt.xlabel('Time (s)')
plt.ylabel('Frequency (Hz)')
plt.tight_layout()
plt.show()

## 5. Spectral Statistics

Let's compute and visualize statistics of the spectral features.

In [ ]:
# Compute statistics of spectral features
spectral_stats = {
    'Centroid': {
        'Mean': np.mean(spectral_centroid),
        'Std Dev': np.std(spectral_centroid),
        'Min': np.min(spectral_centroid),
        'Max': np.max(spectral_centroid),
        'Median': np.median(spectral_centroid)
    },
    'Bandwidth': {
        'Mean': np.mean(spectral_bandwidth),
        'Std Dev': np.std(spectral_bandwidth),
        'Min': np.min(spectral_bandwidth),
        'Max': np.max(spectral_bandwidth),
        'Median': np.median(spectral_bandwidth)
    },
    'Flatness': {
        'Mean': np.mean(spectral_flatness),
        'Std Dev': np.std(spectral_flatness),
        'Min': np.min(spectral_flatness),
        'Max': np.max(spectral_flatness),
        'Median': np.median(spectral_flatness)
    },
    'Rolloff': {
        'Mean': np.mean(spectral_rolloff),
        'Std Dev': np.std(spectral_rolloff),
        'Min': np.min(spectral_rolloff),
        'Max': np.max(spectral_rolloff),
        'Median': np.median(spectral_rolloff)
    }
}

# Display statistics
for feature, stats in spectral_stats.items():
    print(f"\n{feature} Statistics:")
    for stat, value in stats.items():
        print(f"{stat}: {value:.2f}")

In [ ]:
# Create a DataFrame for visualization
stats_df = pd.DataFrame({
    'Centroid': spectral_centroid,
    'Bandwidth': spectral_bandwidth,
    'Flatness': spectral_flatness,
    'Rolloff': spectral_rolloff
})

# Plot distributions of spectral features
plt.figure(figsize=(14, 10))
for i, column in enumerate(stats_df.columns):
    plt.subplot(2, 2, i+1)
    sns.histplot(stats_df[column], kde=True)
    plt.title(f'{column} Distribution')
plt.tight_layout()
plt.show()

## 6. Power Spectrum

Let's compute and visualize the power spectrum of the audio signal.

In [ ]:
# Compute power spectrum
fft_result = np.fft.rfft(audio_data)
fft_magnitude = np.abs(fft_result)
fft_power = fft_magnitude ** 2
fft_frequencies = np.fft.rfftfreq(len(audio_data), 1/sample_rate)

# Plot power spectrum
plt.figure(figsize=(14, 5))
plt.semilogy(fft_frequencies, fft_power)
plt.title('Power Spectrum')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power')
plt.xlim([0, sample_rate/2])
plt.tight_layout()
plt.show()

## 7. Frequency Bands Analysis

Let's analyze the energy in different frequency bands.

In [ ]:
# Define frequency bands
bands = {
    'Sub-bass': (20, 60),
    'Bass': (60, 250),
    'Low-mids': (250, 500),
    'Mids': (500, 2000),
    'High-mids': (2000, 4000),
    'Presence': (4000, 6000),
    'Brilliance': (6000, 20000)
}

# Compute energy in each band
band_energies = {}
for band_name, (low_freq, high_freq) in bands.items():
    # Find indices corresponding to the frequency range
    indices = np.where((fft_frequencies >= low_freq) & (fft_frequencies <= high_freq))[0]
    # Compute energy in the band
    band_energies[band_name] = np.sum(fft_power[indices])

# Normalize energies
total_energy = sum(band_energies.values())
normalized_band_energies = {band: energy / total_energy for band, energy in band_energies.items()}

# Plot band energies
plt.figure(figsize=(14, 5))
plt.bar(normalized_band_energies.keys(), normalized_band_energies.values())
plt.title('Energy Distribution Across Frequency Bands')
plt.xlabel('Frequency Band')
plt.ylabel('Normalized Energy')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 8. Using the Package's Frequency Domain Analysis

Let's use the package's built-in frequency domain analysis function.

In [ ]:
# Use the package's frequency domain analysis function
frequency_analysis_results = analyze_frequency_domain(
    audio_data, 
    sample_rate, 
    title_prefix="Sample Audio"
)

# Display the figures
for fig_name, fig in frequency_analysis_results['figures'].items():
    plt.figure(fig.number)
    plt.tight_layout()
    plt.show()

## Conclusion

In this notebook, we've performed a comprehensive frequency domain analysis of an audio file. We've examined its spectral content using various transforms and features, including STFT, Mel spectrogram, chromagram, and various spectral features like centroid, bandwidth, contrast, flatness, and rolloff. We've also analyzed the energy distribution across different frequency bands.

This analysis provides valuable insights into the frequency characteristics of the audio data, which can be useful for understanding the spectral content of speech signals and for feature extraction in speech recognition tasks.